In [32]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from dataiku.scenario import Trigger


# Accessing the DSS Global/Project Variables
proj = dataiku.Project()
variables = proj.get_variables()

In [33]:
ntile10_dataset = dataiku.Dataset("dpc_covid19_prepared_windows_by_ntile10")
dpc_dataset = dataiku.Dataset("dpc_covid19_prepared")

In [34]:
ntile10_df = ntile10_dataset.get_dataframe()
dpc_df = dpc_dataset.get_dataframe()

In [35]:
ntile10_df.head()

,ntile10,nuovi_positivi_min,nuovi_positivi_max,nuovi_positivi_avg,nuovi_positivi_sum,nuovi_positivi_stddev,count
0,1,78,451,253.000000,21252,84.220022,84
1,2,463,1397,918.452381,77150,294.270362,84
2,3,1400,2818,2035.096386,168913,456.517374,83
3,4,2834,4717,3713.726190,311953,554.433464,84
4,5,4720,7537,5963.337349,494957,817.652847,83


In [36]:
# dpc_df.head()

,data,stato,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,is_holiday,is_weekend
0,2020-02-24 18:00:00+00:00,ITA,101,26,127,94,221,0,221,1,7,NaN,NaN,229,4324,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
1,2020-02-25 18:00:00+00:00,ITA,114,35,150,162,311,90,93,1,10,NaN,NaN,322,8623,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
2,2020-02-26 18:00:00+00:00,ITA,128,36,164,221,385,74,78,3,12,NaN,NaN,400,9587,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
3,2020-02-27 18:00:00+00:00,ITA,248,56,304,284,588,203,250,45,17,NaN,NaN,650,12014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
4,2020-02-28 18:00:00+00:00,ITA,345,64,409,412,821,233,238,46,21,NaN,NaN,888,15695,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False


In [50]:
#Set variables for ntile ranges from ntile_range_dataset in order to compare to full dataset
for num in range(1,11):
    variables["standard"][f'n{num}'] = range(int(ntile10_df['nuovi_positivi_min'][ntile10_df['ntile10'] == num]), int(ntile10_df['nuovi_positivi_max'][ntile10_df['ntile10'] == num]+1))

In [51]:
ntile_dict = {}
for row in dpc_df['nuovi_positivi']:
    for ntile in range(1,11):
        if row in variables["standard"][f'n{ntile}']:
            ntile_dict[row] = int(ntile)

In [52]:
dpc_df['ntile'] = dpc_df['nuovi_positivi'].map(ntile_dict)

/Users/terrijohn/Library/dss/design/code-envs/python/python37_holidays/lib/python3.7/site-packages/pandas/core/indexes/base.py:395: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif issubclass(data.dtype.type, np.bool) or is_bool_dtype(data):


In [48]:
dpc_df[dpc_df['ntile'].isna()]

,data,stato,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,is_holiday,is_weekend,ntile
84,2020-05-18 17:00:00+00:00,ITA,10207,749,10956,55597,66553,-1798,451,127326,32007,NaN,NaN,225886,3041366,1959373.0,pd-IT-0019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,NaN
192,2020-09-03 17:00:00+00:00,ITA,1505,120,1625,27290,28915,1098,1397,208490,35507,230950.0,41962.0,272912,8921658,5342150.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,NaN
297,2020-12-17 17:00:00+00:00,ITA,26427,2855,29282,606061,635343,-10363,18236,1203814,67220,NaN,NaN,1906377,24635720,14246739.0,NaN,183.0,NaN,NaN,NaN,NaN,NaN,NaN,False,False,NaN
302,2020-12-22 17:00:00+00:00,ITA,24948,2687,27635,578320,605955,-7627,13318,1301573,69842,NaN,NaN,1977370,25374719,14386848.0,NaN,201.0,NaN,NaN,NaN,NaN,NaN,NaN,False,False,NaN
453,2021-05-22 17:00:00+00:00,ITA,9488,1430,10918,272826,283744,-8044,4717,3779293,125153,NaN,NaN,4188190,64379095,27568979.0,NaN,64.0,NaN,NaN,4040695.0,147495.0,48308709.0,16070386.0,False,True,NaN
616,2021-11-01 17:00:00+00:00,ITA,2863,364,3227,80495,83722,1274,2818,4558941,132120,NaN,NaN,4774783,104398683,36105689.0,NaN,33.0,NaN,NaN,4595482.0,179301.0,63316846.0,41081837.0,False,False,NaN
687,2022-01-11 17:00:00+00:00,ITA,17067,1677,18744,2115395,2134139,129542,220532,5500938,139559,NaN,NaN,7774863,150147466,43105796.0,NaN,185.0,NaN,NaN,6361711.0,1413152.0,76403257.0,73744209.0,False,False,NaN
785,2022-04-19 17:00:00+00:00,ITA,10214,422,10636,1197643,1208279,-8564,27214,14387830,161893,NaN,NaN,15758002,209450124,55944112.0,NaN,38.0,NaN,NaN,8507229.0,7250773.0,89804639.0,119645485.0,False,False,NaN
795,2022-04-29 17:00:00+00:00,ITA,9942,371,10313,1236804,1247117,-1689,58861,14998689,163377,NaN,NaN,16409183,213390610,56731492.0,NaN,38.0,NaN,NaN,8607251.0,7801932.0,90543721.0,122846889.0,False,False,NaN
826,2022-05-30 17:00:00+00:00,ITA,5281,255,5536,693325,698861,-13187,7537,16531231,166631,NaN,NaN,17396723,220866180,58320769.0,NaN,13.0,NaN,NaN,8755409.0,8641314.0,92311740.0,128554440.0,False,False,NaN


In [49]:
variables

{'standard': {'url': 'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-andamento-nazionale/dpc-covid19-ita-andamento-nazionale.csv',
  'csv_name': 'dpc_covid19_ita_andamento_nazionale_df',
  'min_new_daily_pos': 74,
  'max_new_daily_pos': 220532,
  'n1': range(78, 451),
  'n2': range(463, 1397),
  'n3': range(1400, 2818),
  'n4': range(2834, 4717),
  'n5': range(4720, 7537),
  'n6': range(7548, 13318),
  'n7': range(13331, 18236),
  'n8': range(18255, 27214),
  'n9': range(27354, 58861),
  'n10': range(59555, 220532)},
 'local': {}}

In [0]:
test_variable_set = dataiku.Dataset("test_variable_set")
test_variable_set.write_with_schema(dpc_df)